# Toolbox

In [1]:
from matplotlib.colors import LinearSegmentedColormap
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from optuna.samplers import TPESampler

import shap
import optuna
import catboost as ct
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgbm
import pandas as pd
import sklearn as sk
import xgboost as xgboost

# Read the Data

In [ ]:
# Create a DataFrame
data = pd.read_csv('FILE_PATH\dataset-pkbhx.csv')

In [3]:
# Is there any null values?
print(data.isnull().values.any())

False


# Split the Data

In [3]:
data_y = data[['pkbhx']] # Dataframe with the target
data_X = data.drop(columns=['SMILES', 'pkbhx']) # Dataframe with the features

X_train, X_test, y_train, y_test = model_selection.train_test_split(data_X, data_y, test_size=0.10, random_state=0)

# StandardScaler

In [4]:
sca = StandardScaler()
X_train_sca = pd.DataFrame(
    sca.fit_transform(X_train.drop(['id'], axis=1)),
    columns=X_train.drop(['id'], axis=1).columns
)
X_test_sca = pd.DataFrame(
    sca.transform(X_test.drop(['id'], axis=1)),
    columns=X_test.drop(['id'], axis=1).columns
)

# Hyperparameters Optimization

## Linear Model

In [19]:
reg = LinearRegression().fit(X_train_sca.values, y_train.values.ravel())

In [20]:
y_hat = reg.predict(X_test_sca.values)

In [ ]:
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

## Random Forest

Hold-Out Cross-validation

In [ ]:
# Random Forest Model
def objectiveRF(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse']),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }

    model = RandomForestRegressor(**params, random_state=0)
    model.fit(X_train_sca.values, y_train.values.ravel())
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studyRF = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyRF.optimize(objectiveRF, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyRF.best_params)
print('Best RMSE:', studyRF.best_value)

In [ ]:
model_rf = RandomForestRegressor(random_state=0, n_estimators=393, criterion='absolute_error', max_depth=10, min_samples_split=2, min_samples_leaf=1, max_features='log2')

model_rf.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_rf.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# Random Forest Model
def objectiveRF(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse']),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }

    model = RandomForestRegressor(**params, random_state=0)

    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model,  X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyRF = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyRF.optimize(objectiveRF, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyRF.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyRF.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyRF.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyRF.best_params)
print('Best RMSE:', studyRF.best_value)

In [ ]:
model_rf = RandomForestRegressor(random_state=0, n_estimators=266, criterion='absolute_error', max_depth=10, min_samples_split=3, min_samples_leaf=1, max_features='sqrt')

model_rf.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_rf.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

## XGBoost

Hold-Out Cross-validation

In [ ]:
# XGBoost Model
def objectiveXGB(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
    }

    model = xgboost.XGBRegressor(**params, random_state=0, verbosity=0)
    model.fit(X_train_sca.values, y_train.values.ravel(), verbose=False)
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studyXGB = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyXGB.optimize(objectiveXGB, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyXGB.best_params)
print('Best RMSE:', studyXGB.best_value)

In [ ]:
model_xgb = XGBRegressor(random_state=0, n_estimators=185, learning_rate=0.02596425441970596, 
                         max_depth=10, subsample=0.8969848974548218, colsample_bytree=0.8607583706114562, min_child_weight=5)

model_xgb.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_xgb.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# XGBoost Model
def objectiveXGB(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
    }

    model = xgboost.XGBRegressor(**params, random_state=0, verbosity=0)

    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model,  X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyXGB = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyXGB.optimize(objectiveXGB, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyXGB.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyXGB.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyXGB.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyXGB.best_params)
print('Best RMSE:', studyXGB.best_value)

In [ ]:
model_xgb = XGBRegressor(random_state=0, n_estimators=967, learning_rate=0.010010842387401658, 
                         max_depth=8, subsample=0.29251392417671074, colsample_bytree=0.8129074263707009, min_child_weight=1)

model_xgb.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_xgb.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

## CatBoost

Hold-Out Cross-validation

In [ ]:
# CatBoost Model
def objectiveCT(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

    model = ct.CatBoostRegressor(**params, random_state=0, silent=True, eval_metric='RMSE')
    model.fit(X_train_sca.values, y_train.values.ravel())
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studyCT = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyCT.optimize(objectiveCT, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyCT.best_params)
print('Best RMSE:', studyCT.best_value)

In [ ]:
model_ct = ct.CatBoostRegressor(n_estimators=518, learning_rate=0.049405517082973915, depth=10,
                                l2_leaf_reg=4.134816151740027, grow_policy='Depthwise', random_state=0, silent=True, eval_metric='RMSE')

model_ct.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_ct.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# CatBoost Model
def objectiveCT(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

    model = ct.CatBoostRegressor(**params, random_state=0, silent=True, eval_metric='RMSE')

    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyCT = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyCT.optimize(objectiveCT, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyCT.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyCT.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyCT.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyCT.best_params)
print('Best RMSE:', studyCT.best_value)

In [ ]:
model_ct = ct.CatBoostRegressor(n_estimators=951, learning_rate=0.021075826382483023, depth=10,
                                l2_leaf_reg=4.558823001649244, grow_policy='Depthwise', random_state=0, silent=True, eval_metric='RMSE')

model_ct.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_ct.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

## SVM

Hold-Out Cross-validation

In [ ]:
# SVM Model
def objectiveSVM(trial):
    params = {
        'kernel': trial.suggest_categorical("kernel", ['linear', 'poly', 'rbf', 'sigmoid']),
        'C': trial.suggest_float('C', 1e-3, 5, log=True),
        'degree': trial.suggest_int('degree', 2, 7),
        'gamma': trial.suggest_float('gamma', 1e-3, 0.01, log=True),
        'coef0': trial.suggest_float('coef0', 0.1, 5.0, log=True)
    }

    model = SVR(**params)
    model.fit(X_train_sca.values, y_train.values.ravel())
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studySVM = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studySVM.optimize(objectiveSVM, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studySVM.best_params)
print('Best RMSE:', studySVM.best_value)

In [ ]:
model_svr = SVR(kernel='poly', C=3.7288560084932603, degree=6, gamma=0.0029604914010939115, coef0=3.5821638281807133)

model_svr.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_svr.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# SVM Model
def objectiveSVM(trial):
    params = {
        'kernel': trial.suggest_categorical("kernel", ['linear', 'poly', 'rbf', 'sigmoid']),
        'C': trial.suggest_float('C', 1e-3, 5, log=True),
        'degree': trial.suggest_int('degree', 2, 7),
        'gamma': trial.suggest_float('gamma', 1e-3, 0.01, log=True),
        'coef0': trial.suggest_float('coef0', 0.1, 5.0, log=True)
    }

    model = SVR(**params)
    
    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studySVM = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studySVM.optimize(objectiveSVM, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studySVM.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studySVM.best_trial.user_attrs['mae_scores']
    best_r2_scores = studySVM.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studySVM.best_params)
print('Best RMSE:', studySVM.best_value)

In [ ]:
model_svr = SVR(kernel='rbf', C=4.99744826902557, degree=6, gamma=0.009995777060509877, coef0=0.7149170011195296)

model_svr.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_svr.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

## KNN

Hold-Out Cross-validation

In [ ]:
# Nearest Neighbors Model
def objectiveNN(trial):
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 8, 15),
        'weights': trial.suggest_categorical("weights", ['uniform', 'distance']),
        'metric': trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    }

    model = KNeighborsRegressor(**params)
    model.fit(X_train_sca.values, y_train.values.ravel())
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studyNN = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyNN.optimize(objectiveNN, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyNN.best_params)
print('Best RMSE:', studyNN.best_value)

In [ ]:
model_nn = KNeighborsRegressor(n_neighbors=15, weights='distance', metric='manhattan')

model_nn.fit(X_train_sca.values, y_train.values)
y_hat = model_nn.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test, y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test, y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test, y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test,y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# Nearest Neighbors Model
def objectiveNN(trial):
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 8, 15),
        'weights': trial.suggest_categorical("weights", ['uniform', 'distance']),
        'metric': trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    }

    model = KNeighborsRegressor(**params)
    
    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyNN= optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyNN.optimize(objectiveNN, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyNN.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyNN.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyNN.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyNN.best_params)
print('Best RMSE:', studyNN.best_value)

In [ ]:
model_nn = KNeighborsRegressor(n_neighbors=8, weights='distance', metric='manhattan')

model_nn.fit(X_train_sca.values, y_train.values)
y_hat = model_nn.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test, y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test, y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test, y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test,y_hat)}')

## Decision Tree

Hold-Out Cross-validation

In [ ]:
def objectiveDT(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-3, 0.5, log=True),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 1e-3, 1, log=True)
    }

    model = DecisionTreeRegressor(**params,random_state=0)
    model.fit(X_train_sca.values, y_train.values)
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values, predictions)
    return rmse

studyDT = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyDT.optimize(objectiveDT, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyDT.best_params)
print('Best RMSE:', studyDT.best_value)

In [ ]:
model_dt = DecisionTreeRegressor(random_state=0, max_depth=5, splitter='best', min_samples_split=3,
                                 min_samples_leaf=15, min_weight_fraction_leaf=0.014001513031818463, min_impurity_decrease=0.002364384066963733)

model_dt.fit(X_train_sca.values, y_train.values)
y_hat = model_dt.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test, y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test, y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test, y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test,y_hat)}')

Hold-Out + K-Fold Cross-validation

In [ ]:
# Nearest Neighbors Model
def objectiveDT(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-3, 0.5, log=True),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 1e-3, 1, log=True)
    }

    model = DecisionTreeRegressor(**params,random_state=0)
    
    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyDT= optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyDT.optimize(objectiveDT, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyDT.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyDT.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyDT.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyDT.best_params)
print('Best RMSE:', studyDT.best_value)

In [ ]:
model_dt = DecisionTreeRegressor(random_state=0, max_depth=8, splitter='best', min_samples_split=2,
                                 min_samples_leaf=5, min_weight_fraction_leaf=0.01193235419114749, min_impurity_decrease=0.0010024067265582204)

model_dt.fit(X_train_sca.values, y_train.values)
y_hat = model_dt.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test, y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test, y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test, y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test,y_hat)}')

## MLP

Hold-Out Cross-validation

In [ ]:
def objectiveMLP(trial):
    n_layers = trial.suggest_int('n_layers', 1, 4)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 1, 256))

    # Activation function choice
    activation = trial.suggest_categorical('activation', ['identity', 'tanh', 'logistic', 'relu'])

    # Regularization
    l2_reg = trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True)

    # Solver
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam'])

    model = MLPRegressor(
        random_state=0,
        hidden_layer_sizes=tuple(layers),
        activation=activation, 
        solver=solver,
        alpha=l2_reg,  # L2 regularization parameter
        max_iter=10000000,
    )

    model.fit(X_train_sca.values, y_train.values.ravel())
    predictions = model.predict(X_test_sca.values)
    rmse = root_mean_squared_error(y_test.values.ravel(), predictions)
    return rmse

studyMLP = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyMLP.optimize(objectiveMLP, n_trials=500, n_jobs=1, show_progress_bar=True)

print('Best hyperparameters:', studyMLP.best_params)
print('Best RMSE:', studyMLP.best_value)

In [ ]:
model_MLP = MLPRegressor(
        random_state=0,
        hidden_layer_sizes=[62, 72],
        activation='relu', 
        solver='adam',
        alpha=1.1635523810951026e-05,  # L2 regularization parameter
        max_iter=10000000,
    )

model_MLP.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_MLP.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')

SHapley Additive exPlanations

In [ ]:
explainer = shap.KernelExplainer(model_MLP.predict, X_train_sca.sample(500, random_state=0))
shap_values = explainer.shap_values(X_test_sca)
shap.summary_plot(shap_values, X_test_sca, plot_size=(5, 5), show=False)
plt.show()

Hold-Out + K-Fold Cross-validation

In [ ]:
def objectiveMLP(trial):
    n_layers = trial.suggest_int('n_layers', 1, 4)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 1, 256))

    # Activation function choice
    activation = trial.suggest_categorical('activation', ['identity', 'tanh', 'logistic', 'relu'])

    # Regularization
    l2_reg = trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True)

    # Solver
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam'])

    model = MLPRegressor(
        random_state=0,
        hidden_layer_sizes=tuple(layers),
        activation=activation, 
        solver=solver,
        alpha=l2_reg,  # L2 regularization parameter
        max_iter=10000000,
    )
    
    # Compute MAE and R2 for analysis
    mae_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='r2')

    # Store MAE and R2 scores as user attributes
    trial.set_user_attr('mae_scores', mae_scores)
    trial.set_user_attr('r2_scores', r2_scores)

    # Compute neg_root_mean_squared_error for optimization
    rmse_scores = -cross_val_score(model, X_train_sca.values, y_train.values.ravel(), n_jobs=-1, cv=KFold(n_splits=5, shuffle=True, random_state=0), scoring='neg_root_mean_squared_error')
    trial.set_user_attr('rmse_scores', rmse_scores)

    return rmse_scores.mean()

studyMLP = optuna.create_study(direction='minimize', sampler=TPESampler(seed=0))
studyMLP.optimize(objectiveMLP, n_trials=500, n_jobs=1, show_progress_bar=True)

try:
    best_rmse_scores = studyMLP.best_trial.user_attrs['rmse_scores']
    best_mae_scores = studyMLP.best_trial.user_attrs['mae_scores']
    best_r2_scores = studyMLP.best_trial.user_attrs['r2_scores']

    mean_rmse = np.mean(best_rmse_scores)
    std_rmse = np.std(best_rmse_scores)
    mean_mae = np.mean(best_mae_scores)
    std_mae = np.std(best_mae_scores)
    mean_r2 = np.mean(best_r2_scores)
    std_r2 = np.std(best_r2_scores)

    print("Mean RMSE of Best Model:", mean_rmse)
    print("Standard Deviation of RMSE of Best Model:", std_rmse)
    print("Mean MAE of Best Model:", mean_mae)
    print("Standard Deviation of MAE of Best Model:", std_mae)
    print("Mean R2 of Best Model:", mean_r2)
    print("Standard Deviation of R2 of Best Model:", std_r2)

except ValueError as e:
    print("No completed trials yet.")

print('Best hyperparameters:', studyMLP.best_params)
print('Best RMSE:', studyMLP.best_value)

In [ ]:
model_MLP = MLPRegressor(
        random_state=0,
        hidden_layer_sizes=[198, 20],
        activation='logistic', 
        solver='adam',
        alpha=1.4997590316080432e-05,  # L2 regularization parameter
        max_iter=10000000,
    )

model_MLP.fit(X_train_sca.values, y_train.values.ravel())
y_hat = model_MLP.predict(X_test_sca.values)
print(f'R**2: {r2_score(y_test.values.ravel(), y_hat)}')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_test.values.ravel(), y_hat)}')
print(f'Mean square error (MSE): {mean_squared_error(y_test.values.ravel(), y_hat)}')
print(f'Root Mean Square error (RMSE): {root_mean_squared_error(y_test.values.ravel(),y_hat)}')